<a href="https://colab.research.google.com/github/williambiondi/RecommenderGS/blob/main/DataCleaningGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install altair -q
!pip install altair_viewer -q
!pip install requests -q

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import re

In [ ]:
items_df = pd.read_excel("OrdiniGranSasso.xlsx")
purchases_df = pd.read_excel("OrdiniGranSassoDescrizione.xlsx")

#Pulizia dei dati


In [ ]:
purchases_df.dropna(subset=['DescrizioneEstesaInglese'], inplace=True)

In [ ]:
currencies = purchases_df['CustomerCurrencyCode']
currencies.drop_duplicates(inplace=True)

In [ ]:
def exchange_rate(currency):
  KEY = 'MYKEY'
  url = f'http://data.fixer.io/api/latest?access_key={KEY}'
  try:
    response = requests.get(url)
    data = response.json()
    exchange_rate = float(data['rates'][currency])
    return 1/exchange_rate
  except requests.RequestException as e:
    print("error")
    return None

In [ ]:
rates = {}
for currency in currencies:
  rates[currency] = exchange_rate(currency)

In [ ]:
rates

{'EUR': 1.0,
 'NOK': 0.08499454462515324,
 'USD': 0.9287551894196209,
 'CAD': 0.6832346514751719,
 'CHF': 1.0570668087364457,
 'GBP': 1.1659278570479175,
 'TRY': 0.03202691695830377,
 'AUD': 0.6087801932146578,
 'JPY': 0.006354417051879901}

In [ ]:
def to_euro(row, column):
  rate = rates.get(row['CustomerCurrencyCode'], 1)
  return round(row[column] * rate,2)

In [ ]:
purchases_df['UnitPriceTaxEUR'] = purchases_df.apply(lambda x: to_euro(x, 'UnitPriceInclTax'), axis=1)
purchases_df['RefundedEUR'] = purchases_df.apply(lambda x: to_euro(x, 'RefundedAmount'), axis=1)

In [ ]:
purchases_selected = purchases_df[['Id', 'CustomerId', 'CreatedOnUtc', 'RefundedEUR','UnitPriceTaxEUR','Sku']]

In [ ]:
customers_selected = purchases_df[['CustomerId', 'Name']]
customers_selected.drop_duplicates(subset='CustomerId', inplace=True)
customers_selected = customers_selected.reset_index(drop=True)
customers_selected.rename(columns={'Name':'Country'}, inplace=True)

<ipython-input-24-19dfc760e453>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_selected.drop_duplicates(subset='CustomerId', inplace=True)


In [ ]:
items_df = pd.merge(purchases_df, items_df, on='Sku', how='inner')

In [ ]:
items_df.rename(columns={'Sku_x':'Sku', 'UnitPriceTaxEUR':'UnitPrice'}, inplace=True)

In [ ]:
items_selected = items_df[['Sku', 'LocaleValue', 'Categoria','DescrizioneEstesaInglese','UnitPrice']].drop_duplicates('Sku')
items_selected.dropna(subset=['DescrizioneEstesaInglese'], inplace=True)
items_selected = items_selected.reset_index(drop=True)

In [ ]:
items_selected.head()

,Sku,LocaleValue,Categoria,DescrizioneEstesaInglese,UnitPrice
0,A20205716614238589,MICRO RHOMBUS STITCH CREW NECK,NaN,Crew neck sweater in pure merino wool with mic...,130.0
1,A20201312414230005,WAVE PATTERN TURTLENECK,NaN,"Turtleneck sweater in pure merino wool, embell...",160.0
2,A20205711414241046,SALT AND PEPPER STITCH FULL ZIP,NaN,Cardigan in pure extra fine merino wool with t...,180.0
3,A20201312022612852,AIR WOOL ARGYLE CREW NECK,NaN,3-ply moulinè effect Air Wool crew neck sweate...,145.0
4,A20201314022615116,AIR WOOL ARGYLE V NECK,NaN,<p>Argyle Air Wool V-neck sweater. The special...,145.0


In [ ]:
item_map = dict()
for i in range(items_selected.shape[0]):
  item_map[i] = items_selected.loc[i,'Sku']
items_selected['ItemId'] = item_map.keys()
inverted_item_map = {v: k for k, v in item_map.items()}

In [ ]:
user_map = dict()
for i in range(customers_selected.shape[0]):
  user_map[i] = customers_selected.loc[i,'CustomerId']
customers_selected['UserId'] = user_map.keys()
inverted_uid_map = {v: k for k, v in user_map.items()}

Recupero informazioni da Sku

In [ ]:
items_selected['Collection'] = ['Fall-Winter' if sku[0]=='A' else 'Spring-Summer' for sku in items_selected['Sku']]
items_selected['Availability'] = [1 if sku[1:5]=='2023' else 0 for sku in items_selected['Sku']]
items_selected['Color'] = [sku[15:18] for sku in items_selected['Sku']]
items_selected['Gender'] = ['Women' if int(sku[7]) == 2 else 'Men' for sku in items_selected['Sku']]

Recupero categorie e composizione dalla descrizione

In [ ]:
def clean_description(description):
  clean = re.sub(r'<[^>]+>','', description.lower().strip())
  return re.sub(r'[^\w\s]','',clean)

In [ ]:
def get_composition(description, composition_list):
  for word in description.split(" "):
    if word.lower() in composition_list:
      return word.lower()

In [ ]:
composition_list = ['alcantara', 'cashmere','wool', 'cotton', 'nylon', 'viscose', 'eco-feather', 'silk', 'merinos', 'merinowool', 'linen']

In [ ]:
category_set = set()
for category in items_selected['Categoria']:
  if not pd.isna(category):
    for item in category.split(";"):
      category_set.add(item)
  else:
    continue
category_list = list(category_set)

In [ ]:
items_selected['Description'] = items_selected['DescrizioneEstesaInglese'].apply(clean_description)
items_selected.drop(columns=['DescrizioneEstesaInglese'], inplace=True)
items_selected['Composition'] = items_selected['Description'].apply(lambda x: get_composition(x, composition_list))

In [ ]:
wool_map = {'merinos':'wool', 'merinowool': 'wool'}
items_selected['Composition']= items_selected['Composition'].map(lambda x: wool_map.get(x,x))

In [ ]:
items_selected

In [ ]:
category_list

['Pantaloni e Bermuda',
 'Polo, Camicie e T-shirt',
 'Polo, Camicie e Top',
 'Abiti, Pantaloni e Gonne',
 'I classici',
 'Pantaloni',
 'Maglieria, Giacche',
 'Beachwear',
 'Pantaloni e Gonne',
 'Cashmere',
 'Sciarpe e Cappelli',
 'Maglieria',
 'Vintage',
 'Capispalla']

In [ ]:
categories = {
    "Pantaloni e Bermuda": {"pant", "bermuda", "trousers", "shorts"},
    "Cashmere": {"cashmere"},
    "Beachwear": {"beachwear", "swimwear", "bikini", "swimsuit", "swim"},
    "Maglieria": {"knit", "sweater", "cardigan", "wool", "crew", "neck", "crewneck", "turtleneck", "lana"},
    "Pantaloni e Gonne": {"pant", "skirt"},
    "I classici": {"classic", "traditional", "timeless", "waistcoat"},
    "Polo, Camicie e T-shirt": {"polo", "shirt", "t-shirt", "tshirt", "tee", "tracksuit", "sweatshirt"},
    "Abiti, Pantaloni e Gonne": {"dress", "pant", "skirt"},
    "Sciarpe e Cappelli": {"scarf", "hat", "cap"},
    "Capispalla": {"coat", "jacket", "outerwear", "faux", "vest","blouse"},
    "Vintage": {"vintage", "retro", "old-school"},
    "Pantaloni": {"pant","pants", "trousers", "slacks", "joggers"},
    "Giacche": {"jacket", "cardigan"},
    "Polo, Camicie e Top": {"polo", "shirt", "top","tank"}
}

In [ ]:
def get_category(description, categories):
  description = set(description.split(" ")[:20])
  matched = list()
  for category, keywords in categories.items():
    if description.intersection(keywords):
      matched.append(category)
  return matched

In [ ]:
items_selected.rename(columns={'LocaleValue':'ItemName','Categoria':'Category'}, inplace=True)

In [ ]:
for idx, row in items_selected.iterrows():
  if pd.isna(row['Category']):
    predicted = get_category(row['Description'], categories)
    if predicted:
      items_selected.at[idx,'Category'] = predicted
  else:
    items_selected.at[idx,'Category'] = row['Category'].split(";")

In [ ]:
purchases_selected['UserId'] = purchases_selected['CustomerId'].map(inverted_uid_map)
purchases_selected['ItemId'] = purchases_selected['Sku'].map(inverted_item_map)

<ipython-input-46-9e537e27da12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_selected['UserId'] = purchases_selected['CustomerId'].map(inverted_uid_map)
<ipython-input-46-9e537e27da12>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_selected['ItemId'] = purchases_selected['Sku'].map(inverted_item_map)


In [ ]:
purchases_selected = purchases_selected[['Id','UserId','ItemId','CreatedOnUtc', 'RefundedEUR', 'UnitPriceTaxEUR']]

In [ ]:
items_selected.to_csv('ArticoliGransasso.csv')
purchases_selected.to_csv('OrdiniGransasso.csv')
customers_selected.to_csv('ClientiGransasso.csv')

In [ ]:
items_selected = pd.read_csv('ArticoliGransasso.csv')
customers_selected = pd.read_csv('ClientiGransasso.csv')
purchases_selected = pd.read_csv('OrdiniGransasso.csv')

items_selected.drop(columns=['Unnamed: 0'], inplace=True)
customers_selected.drop(columns=['Unnamed: 0'], inplace=True)
purchases_selected.drop(columns=['Unnamed: 0'], inplace=True)

#Analitiche

In [ ]:
purchases_selected.head()

,Id,UserId,ItemId,CreatedOnUtc,RefundedEUR,UnitPriceTaxEUR,Interaction
0,8812,0,0,2021-01-01 09:29:05.2513839,0.0,130.0,1
1,8813,1,1,2021-01-01 09:39:57.3829465,0.0,160.0,1
2,8813,1,2,2021-01-01 09:39:57.3829465,0.0,180.0,1
3,8814,2,3,2021-01-01 10:23:40.5132880,0.0,145.0,1
4,8814,2,4,2021-01-01 10:23:40.5132880,0.0,145.0,1


In [ ]:
price_chart = alt.Chart(items_selected).mark_bar().encode(
    alt.X('UnitPrice:Q', bin=alt.Bin(step=150), title='Prezzo in EUR'),
    alt.Y('count()', title='Numero Articoli', scale=alt.Scale(type='log')),
    tooltip=[alt.Tooltip('PriceRange', title='Fascia di Prezzo'), alt.Tooltip('count()', title='Numero Articoli')]
).properties(title='Distribuzione prezzi')
alt.hconcat(price_chart)

alt.HConcatChart(...)

In [ ]:
total_per_order = purchases_selected.merge(items_selected, on='ItemId').groupby('Id')['UnitPrice'].sum().reset_index()

In [ ]:
bins = list(range(0, int(items_selected['UnitPrice'].max()) + 500, 500))
total_per_order['Range']=pd.cut(items_selected['UnitPrice'], bins = bins, right=False).astype(str)

In [ ]:
total_per_order.describe()

,Id,UnitPrice
count,10777.000000,10777.000000
mean,14231.242739,444.997471
std,3129.047603,507.468425
min,8812.000000,0.000000
25%,11525.000000,200.000000
50%,14228.000000,307.450000
75%,16945.000000,525.000000
max,19650.000000,16918.260000


In [ ]:
price_chart = alt.Chart(total_per_order).mark_bar().encode(
    alt.X('UnitPrice:Q', bin=alt.Bin(step=500), title='Prezzo in EUR'),
    alt.Y('count()', title='Ordini'),
    tooltip=[alt.Tooltip('Range', title='Fascia di Prezzo'), alt.Tooltip('count()', title='Numero Articoli')]
).properties(title='Distribuzione spese per singolo ordine')
alt.hconcat(price_chart)

alt.HConcatChart(...)

In [ ]:
order_customer = purchases_selected.merge(customers_selected, on='UserId')
full = order_customer.merge(items_selected, on='ItemId')
full.head()
orders = full.groupby(['Id','Country'])['UnitPrice'].sum().reset_index()
mean_order_per_country = full.groupby(['Country'])['UnitPrice'].mean().reset_index()

In [ ]:
nations_filter = alt.selection_multi(fields=["Country"])
nations_chart = alt.Chart(orders).mark_bar().encode(
    x="count()",
    y=alt.Y("Country:N"),
    color=alt.condition(nations_filter,
        alt.Color("Name:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).properties(title='Ordini per nazione',selection=nations_filter)
nations_chart_mean_expense = alt.Chart(mean_order_per_country).mark_bar().encode(
    x="UnitPrice:Q",
    y=alt.Y("Country:N"),
    color=alt.condition(nations_filter,
        alt.Color("Country:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).properties(title='Spesa media per singolo ordine per nazione',selection=nations_filter)
alt.hconcat(nations_chart,nations_chart_mean_expense)

alt.HConcatChart(...)